# Demonstrate Seq2Seq Wrapper with twitter chat log

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [2]:
import tensorflow as tf
import numpy as np
from google.colab import drive
print(tf.__version__)
drive.mount('/content/drive')
#with open('/content/drive/My Drive/foo.txt', 'w') as f:
#  f.write('Hello Google Drive!')
#drive.flush_and_unmount()
print('Check the todo.txt file context is right.....')
!cat /content/drive/My\ Drive/ML/practical_seq2seq/todo.txt
import sys
sys.path.append('/content/drive/My Drive/ML/practical_seq2seq')
#sys.path.insert(0, '/content/drive/My Drive/ML/practical_seq2seq')

# preprocessed data
from datasets.twitter import data
import data_utils

1.15.2
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoobas&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.activity.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fexperimentsandconfigs%20https%3a%2f%2fwww.googleapis.com%2fauth%2fphotos.native&response_type=code

Enter your authorization code:
4/1AY0e-g7EasffsfO5A6GUY98CIrAW2I9cTEKLcc0bPOKKsjnPIFYs8EEcwUun45LtGSA
Mounted at /content/drive
Check the todo.txt file context is right.....
1.seems we can just comments out the train function becasue we already have a model file
2.we can try to just run it can see the result ok or not@20210407


In [3]:
# load data from pickle and npy files
metadata, idx_q, idx_a = data.load_data(PATH='/content/drive/My Drive/ML/practical_seq2seq/datasets/twitter/')
(trainX, trainY), (testX, testY), (validX, validY) = data_utils.split_dataset(idx_q, idx_a)

In [20]:
# parameters 
xseq_len = trainX.shape[-1]
yseq_len = trainY.shape[-1]
batch_size = 16
xvocab_size = len(metadata['idx2w'])  
yvocab_size = xvocab_size
emb_dim = 1024

In [21]:
import seq2seq_wrapper

In [28]:
import importlib
importlib.reload(seq2seq_wrapper)

<module 'seq2seq_wrapper' from '/content/drive/My Drive/ML/practical_seq2seq/seq2seq_wrapper.py'>

In [29]:
model = seq2seq_wrapper.Seq2Seq(xseq_len=xseq_len,
                               yseq_len=yseq_len,
                               xvocab_size=xvocab_size,
                               yvocab_size=yvocab_size,
                               ckpt_path='ckpt/twitter/',
                               emb_dim=emb_dim,
                               num_layers=3
                               )

<log> Building Graph 

TypeError: ignored

In [ ]:
val_batch_gen = data_utils.rand_batch_gen(validX, validY, 256)
test_batch_gen = data_utils.rand_batch_gen(testX, testY, 256)
train_batch_gen = data_utils.rand_batch_gen(trainX, trainY, batch_size)

In [ ]:
sess = model.train(train_batch_gen, val_batch_gen)

In [ ]:
sess = model.restore_last_session()

In [ ]:
input_ = test_batch_gen.__next__()[0]
output = model.predict(sess, input_)
print(output.shape)

In [ ]:
replies = []
for ii, oi in zip(input_.T, output):
    q = data_utils.decode(sequence=ii, lookup=metadata['idx2w'], separator=' ')
    decoded = data_utils.decode(sequence=oi, lookup=metadata['idx2w'], separator=' ').split(' ')
    if decoded.count('unk') == 0:
        if decoded not in replies:
            print('q : [{0}]; a : [{1}]'.format(q, ' '.join(decoded)))
            replies.append(decoded)